In [1]:
import csv
import pandas as pd
import io
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn import impute

import keras_tuner
import autokeras as ak

In [11]:
csv_file = "E:\BANGKIT2022\CapstoneProject\data_gathering\data\cleaned_data.csv"

dataframe = pd.read_csv(csv_file, delimiter=';')
dataframecopy = pd.read_csv(csv_file, delimiter=';')

In [12]:
dataframe.head()

,Food_Name,Energi,Protein,Karbohidrat_total,Lemak_total
0,Heavenly Blush,90.0,2.0,15.0,2.5
1,A&W,170.0,0.0,43.0,0.0
2,A&W Beef burger,342.0,13.0,39.0,16.0
3,A&W Burger cheeseburger,398.0,15.0,39.0,21.0
4,A&W burger mozza,550.0,25.0,43.0,33.0


In [13]:
dataframe.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Food_Name          839 non-null    object 
 1   Energi             839 non-null    float64
 2   Protein            839 non-null    float64
 3   Karbohidrat_total  839 non-null    float64
 4   Lemak_total        839 non-null    float64
dtypes: float64(4), object(1)
memory usage: 32.9+ KB


In [14]:
print(dataframe.describe())

            Energi     Protein  Karbohidrat_total  Lemak_total
count   839.000000  839.000000         839.000000   839.000000
mean    244.092718    8.104756          27.286293    10.269642
std     236.195633   12.977977          31.643319    14.536275
min       0.000000    0.000000           0.000000     0.000000
25%     100.000000    1.060000           7.850000     2.000000
50%     163.000000    4.000000          20.000000     6.000000
75%     330.000000    9.000000          40.000000    13.645000
max    1914.000000  170.000000         640.000000   148.000000


In [15]:
dataframe['Energi'] = dataframe['Energi'].astype('float')
labels = pd.factorize(dataframe.pop('Food_Name'))[0].astype(np.int64)

In [16]:
numeric_columns = dataframe.select_dtypes(include=['int64','float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy="constant", fill_value=0)),
    ('scaler', preprocessing.MinMaxScaler())
])

print("Number of numerical columns:", len(numeric_columns))
print("Total number of columns in the feature dataframe:", dataframe.shape[1])

Number of numerical columns: 4
Total number of columns in the feature dataframe: 4


In [17]:
preprocessor = compose.ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_columns),
])

# Display the shapes of the training dataset for final inspection
array_features_train = preprocessor.fit_transform(dataframe)
print("Transformed features from df_features_train.shape: {} to array_features_train.shape: {}".format(dataframe.shape, array_features_train.shape))

Transformed features from df_features_train.shape: (839, 4) to array_features_train.shape: (839, 4)


In [18]:
label_encoder = preprocessing.LabelEncoder()
array_target_train = label_encoder.fit_transform(labels)
print("array_features_train.shape: {} array_target_train.shape: {}".format(array_features_train.shape, array_target_train.shape))

array_features_train.shape: (839, 4) array_target_train.shape: (839,)


In [11]:
start_time_module = datetime.now()

auto_model = ak.StructuredDataClassifier(loss='categorical_crossentropy',
                                         metrics=['accuracy'],
                                         max_trials=50,
                                         objective=keras_tuner.Objective("accuracy", direction="max")
                                        )
auto_model.fit(x=array_features_train,
               y=array_target_train,
               epochs=100,
               batch_size=32,
               verbose=1)
print('Total time for model fitting:', (datetime.now() - start_time_module))

Trial 50 Complete [00h 00m 14s]
accuracy: 0.08380682021379471

Best accuracy So Far: 0.9389204382896423
Total elapsed time: 00h 11m 00s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
27/27 [==============================] - 1s 12ms/step - loss: 8.3031 - accuracy: 0.0000e+00
Epoch 2/100
27/27 [==============================] - 0s 12ms/step - loss: 4.8883 - accuracy: 0.0656
Epoch 3/100
27/27 [==============================] - 0s 12ms/step - loss: 3.6307 - accuracy: 0.2348
Epoch 4/100
27/27 [==============================] - 0s 12ms/step - loss: 2.8649 - accuracy: 0.4327
Epoch 5/100
27/27 [==============================] - 0s 12ms/step - loss: 2.2649 - accuracy: 0.6174
Epoch 6/100
27/27 [==============================] - 0s 11ms/step - loss: 1.8006 - accuracy: 0.7521
Epoch 7/100
27/27 [==============================] - 0s 11ms/step - loss: 1.4315 - accuracy: 0.8367
Epoch 8/100
27/27 [==============================] - 0s 12ms/step - loss: 1.1678 - accuracy: 0.8856
Epoch 9/100
27/27 [===

In [12]:
final_model = auto_model.export_model()
final_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 4)                0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 4)                9         
 n)                                                              
                                                                 
 dense (Dense)               (None, 512)               2560      
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                             

In [13]:
final_model.save('saved_model/testing_model_2.h5')

In [2]:
final_model = load_model('./saved_model/testing_model_2.h5')
final_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 4)                0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 4)                9         
 n)                                                              
                                                                 
 dense (Dense)               (None, 512)               2560      
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                             

In [34]:
array = [[400.0,15.0,25.0,17.0]]
test_data = preprocessor.transform(pd.DataFrame(data=array, index=np.arange(len(array)), columns=['Energi','Protein','Karbohidrat_total','Lemak_total']))
test_prediction = (-loaded_model.predict(test_data)).argsort()

In [35]:
print(test_prediction[0][:5])

[166   2 710 418 449]


In [30]:
name_labels = dataframecopy['Food_Name'].to_list()

In [31]:
located_data = dataframecopy.loc[dataframecopy['Food_Name'] == 'A&W Beef burger']
print([located_data['Energi'].values[0],located_data['Protein'].values[0],located_data['Karbohidrat_total'].values[0],located_data['Lemak_total'].values[0]])

[342.0, 13.0, 39.0, 16.0]


In [36]:
for i in test_prediction[0][:5]:
    print(array)
    print(dataframe.loc[i,:])
    print(name_labels[i])

[[400.0, 15.0, 25.0, 17.0]]
Energi               370.0
Protein               16.0
Karbohidrat_total     22.0
Lemak_total           17.0
Name: 166, dtype: float64
Domino Pizza Meatzza
[[400.0, 15.0, 25.0, 17.0]]
Energi               342.0
Protein               13.0
Karbohidrat_total     39.0
Lemak_total           16.0
Name: 2, dtype: float64
A&W Beef burger
[[400.0, 15.0, 25.0, 17.0]]
Energi               250.0
Protein                9.0
Karbohidrat_total     23.0
Lemak_total            8.0
Name: 710, dtype: float64
Starbucks iced caffe mocha (whole milk
[[400.0, 15.0, 25.0, 17.0]]
Energi               300.0
Protein               15.0
Karbohidrat_total     20.0
Lemak_total           12.0
Name: 418, dtype: float64
Mcdonald Cheeseburger
[[400.0, 15.0, 25.0, 17.0]]
Energi               350.0
Protein               10.0
Karbohidrat_total     40.0
Lemak_total           13.0
Name: 449, dtype: float64
Mie Sedaap selection korean sicy soup


In [32]:
model = tf.keras.Sequential([
  layers.CategoryEncoding(num_tokens=4),
  layers.Normalization(),
  layers.Dense(512, activation='relu'),
  layers.BatchNormalization(),
  layers.Dense(1024, activation='relu'),
  layers.BatchNormalization(),
  layers.Dense(len(dataframe), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00000001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x=array_features_train,
          y=array_target_train,
          epochs=500)

Epoch 1/500
27/27 [==============================] - 1s 8ms/step - loss: 7.1493 - accuracy: 0.0000e+00
Epoch 2/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1527 - accuracy: 0.0000e+00
Epoch 3/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1391 - accuracy: 0.0012
Epoch 4/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1508 - accuracy: 0.0000e+00
Epoch 5/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1425 - accuracy: 0.0000e+00
Epoch 6/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1677 - accuracy: 0.0012
Epoch 7/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1565 - accuracy: 0.0000e+00
Epoch 8/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1373 - accuracy: 0.0000e+00
Epoch 9/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1609 - accuracy: 0.0000e+00
Epoch 10/500
27/27 [==============================] - 0s 7ms/step - loss: 7.1388 

KeyboardInterrupt: 

Retrain models with augmented data

In [2]:
csv_file = "E:\BANGKIT2022\CapstoneProject\data_gathering\data\cleaned_data.csv"

dataframe1 = pd.read_csv(csv_file, delimiter=';')
dataframe2 = pd.read_csv(csv_file, delimiter=';')

In [4]:
dataframe1.pop("Food_Name")
dataframe2.pop("Food_Name")

0                                Heavenly Blush
1                                           A&W
2                               A&W Beef burger
3                       A&W Burger cheeseburger
4                              A&W burger mozza
                         ...                   
834    Yoshinoya Veggie beef bowl (Reguler bowl
835                       You C100 Orange water
836                     You C1000 vitamin apple
837                     You C1000 vitamin lemon
838                    You C1000 vitamin orange
Name: Food_Name, Length: 839, dtype: object

In [7]:
for i in dataframe1.columns:
    dataframe1[i] = dataframe1[i].apply(lambda x:x*0.9)
    dataframe2[i] = dataframe2[i].apply(lambda x:x*1.1)

In [9]:
dataframe1['Energi'] = dataframe1['Energi'].astype('float')
dataframe2['Energi'] = dataframe2['Energi'].astype('float')

In [19]:
array_features_train1 = preprocessor.fit_transform(dataframe1)
array_features_train2 = preprocessor.fit_transform(dataframe2)

In [21]:
loaded_model = load_model('./saved_model/testing_model_2.h5')

In [27]:
loaded_model.compile(loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

loaded_model.fit(x=array_features_train2,
               y=array_target_train,
               epochs=300,
               batch_size=32,
               verbose=1)

Epoch 1/300
27/27 [==============================] - 1s 10ms/step - loss: 1.6036 - accuracy: 0.4875
Epoch 2/300
27/27 [==============================] - 0s 10ms/step - loss: 1.3028 - accuracy: 0.5876
Epoch 3/300
27/27 [==============================] - 0s 10ms/step - loss: 1.1266 - accuracy: 0.5971
Epoch 4/300
27/27 [==============================] - 1s 24ms/step - loss: 1.3082 - accuracy: 0.5244
Epoch 5/300
27/27 [==============================] - 1s 45ms/step - loss: 1.3152 - accuracy: 0.5542
Epoch 6/300
27/27 [==============================] - 0s 11ms/step - loss: 1.2303 - accuracy: 0.5542
Epoch 7/300
27/27 [==============================] - 0s 10ms/step - loss: 1.5366 - accuracy: 0.4982
Epoch 8/300
27/27 [==============================] - 0s 10ms/step - loss: 1.3687 - accuracy: 0.5435
Epoch 9/300
27/27 [==============================] - 0s 10ms/step - loss: 1.2130 - accuracy: 0.5864
Epoch 10/300
27/27 [==============================] - 0s 10ms/step - loss: 1.4019 - accuracy: 0.5423

In [38]:
loaded_model.save('saved_model/testing_model_3.h5')